Question 3
3.1 Using the provided dataset, preprocess the data with CountVectorizer and train a Naive Bayes classifier to predict whether a given command should be classified as "Allow" or "Deny". 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report

In [2]:
data = {
    "spoken_phrase": ['grant access', 'deny access', 'approve access', 'block access', 'let in'],
    "action": ['Allow', 'Deny', 'Allow', 'Deny', 'Allow']
}

In [ ]:
df = pd.DataFrame(data)

# selecting features and label
X = df['spoken_phrase']
y = df['action']

"""# applying countervectoriser
vectorizer = CountVectorizer()
X_vec = vectorizer.fit_transform(X).toarray()"""

# tf-idf vectorizer
tf = TfidfVectorizer()
X_vec = tf.fit_transform(X).toarray()

print(f"X vectorized shape: {X_vec.shape}")


X vectorized shape: (5, 7)


In [10]:
# creating model 
# gaussian model
gnb = GaussianNB()
gnb.fit(X_vec, y)

gnb_y_pred = gnb.predict(X_vec)
print(f"GaussianNB predictions: {gnb_y_pred}")

accuracy = accuracy_score(y, gnb_y_pred)
print(f"GaussianNB accuracy: {accuracy*100:.2f}%")

"""# multinomial model
mnb = MultinomialNB()
mnb.fit(X_vec, y)
mnb_y_pred = mnb.predict(X_vec)
print(f"MultinomialNB predictions: {mnb_y_pred}")"""


GaussianNB predictions: ['Allow' 'Deny' 'Allow' 'Deny' 'Allow']
GaussianNB accuracy: 100.00%


'# multinomial model\nmnb = MultinomialNB()\nmnb.fit(X_vec, y)\nmnb_y_pred = mnb.predict(X_vec)\nprint(f"MultinomialNB predictions: {mnb_y_pred}")'

3.2 Predict the intent of a new command: "block entry"

Also, calculate and display the probability scores for each class (Allow/Deny). 

In [15]:
# tesing the model
def predict_command(command):
    phrase_vec = vectorizer.transform([command]).toarray()
    pred = gnb.predict(phrase_vec)[0]
    prob = gnb.predict_proba(phrase_vec)[0]
    return pred, prob

In [16]:
# test input data
command = 'block entry'
result, probs = predict_command(command)

print(f"Test phrase: '{command}'\n Prediction: {result}\n Probabilities: {probs}")

Test phrase: 'block entry'
 Prediction: Deny
 Probabilities: [1.1993559e-14 1.0000000e+00]


In [ ]:
import rospy 
from std_msgs.msg import String

In [ ]:
def init_ros():
    # initialize ROS node
    rospy.init_node('voice_gate_controller', anonymous=True)
    publisher = rospy.Publisher('/gate_command', String, queue_size=10)
    return publisher

In [ ]:
def pub_gate_command(punlisher, command, confidence):
    # publishing the gate command to ROS topic
    message = String() # creating a message
    message.data = f"{command}:{confidence:.3f}"

    # publish to gate commandtopic
    publisher.publish(message)
    rospy.loginfo(f"Published to /gate_command: {message.data}")

In [ ]:
def voice_to_gate(spoken_phrase):
    # predicting intent using the model
    pred, prob = predict_command(spoken_phrase)
    confidence = max(prob)
    # initializing ROS
    try: 
        if not rospy.get_node_url():
            publisher=init_ros()
        else:
            publisher=rospy.Publisher('/gate_command', String, queue_size=10)
    except:
        publisher=init_ros()

    # publish command to gate control system
    pub_gate_command(publisher, pred, confidence)

    # logging interactions
    print(f"Voice command: '{spoken_phrase}")
    print(f"Prediction: {pred} (confidence: {confidence:.3f})")
    print(f"Sent to gate: /gate_command -> {pred}:{confidence:.3f}")

    return pred, confidence